# Wave Equation: Vibrating String Simulation

## Theoretical Background

The one-dimensional wave equation describes the transverse displacement $u(x, t)$ of a vibrating string fixed at both ends. This fundamental partial differential equation governs wave propagation in numerous physical systems.

### The Wave Equation

The classical wave equation in one spatial dimension is given by:

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}$$

where:
- $u(x, t)$ is the displacement of the string at position $x$ and time $t$
- $c = \sqrt{T/\mu}$ is the wave speed
- $T$ is the tension in the string
- $\mu$ is the linear mass density

### Boundary and Initial Conditions

For a string of length $L$ fixed at both ends:

**Boundary Conditions (Dirichlet):**
$$u(0, t) = 0 \quad \text{and} \quad u(L, t) = 0 \quad \forall t \geq 0$$

**Initial Conditions:**
$$u(x, 0) = f(x) \quad \text{(initial displacement)}$$
$$\frac{\partial u}{\partial t}(x, 0) = g(x) \quad \text{(initial velocity)}$$

### Analytical Solution via Separation of Variables

The general solution can be expressed as a Fourier series:

$$u(x, t) = \sum_{n=1}^{\infty} \left[ A_n \cos\left(\frac{n\pi c t}{L}\right) + B_n \sin\left(\frac{n\pi c t}{L}\right) \right] \sin\left(\frac{n\pi x}{L}\right)$$

where the coefficients are determined by:

$$A_n = \frac{2}{L} \int_0^L f(x) \sin\left(\frac{n\pi x}{L}\right) dx$$

$$B_n = \frac{2}{n\pi c} \int_0^L g(x) \sin\left(\frac{n\pi x}{L}\right) dx$$

### Numerical Approach: Finite Difference Method

We discretize space and time using a uniform grid with spacing $\Delta x$ and $\Delta t$. The second-order central difference approximation yields:

$$\frac{u_i^{n+1} - 2u_i^n + u_i^{n-1}}{(\Delta t)^2} = c^2 \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{(\Delta x)^2}$$

Solving for $u_i^{n+1}$:

$$u_i^{n+1} = 2u_i^n - u_i^{n-1} + r^2 (u_{i+1}^n - 2u_i^n + u_{i-1}^n)$$

where $r = c \Delta t / \Delta x$ is the Courant number. For stability, we require the **CFL condition**:

$$r \leq 1 \quad \Rightarrow \quad \Delta t \leq \frac{\Delta x}{c}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Physical parameters
L = 1.0          # Length of string (m)
c = 1.0          # Wave speed (m/s)
T_final = 2.0    # Total simulation time (s)

# Numerical parameters
Nx = 100         # Number of spatial points
dx = L / (Nx - 1)
dt = 0.5 * dx / c  # CFL condition with safety factor
Nt = int(T_final / dt) + 1
r = c * dt / dx    # Courant number

print(f"Spatial grid points: {Nx}")
print(f"Time steps: {Nt}")
print(f"Courant number r = {r:.4f}")
print(f"CFL condition satisfied: {r <= 1}")

In [ ]:
# Spatial grid
x = np.linspace(0, L, Nx)

# Initial conditions
def initial_displacement(x):
    """Plucked string: triangular initial shape"""
    x0 = 0.3 * L  # Pluck position
    A = 0.1       # Amplitude
    return np.where(x <= x0, 
                    A * x / x0, 
                    A * (L - x) / (L - x0))

def initial_velocity(x):
    """String initially at rest"""
    return np.zeros_like(x)

# Initialize solution arrays
u_prev = initial_displacement(x)  # u^{n-1}
u_curr = np.copy(u_prev)          # u^n
u_next = np.zeros(Nx)             # u^{n+1}

# First time step using initial velocity
# u^1 = u^0 + dt * g(x) + 0.5 * r^2 * (u_{i+1}^0 - 2*u_i^0 + u_{i-1}^0)
v0 = initial_velocity(x)
u_curr[1:-1] = (u_prev[1:-1] + dt * v0[1:-1] + 
               0.5 * r**2 * (u_prev[2:] - 2*u_prev[1:-1] + u_prev[:-2]))

# Store solutions for visualization
u_history = [u_prev.copy(), u_curr.copy()]
time_points = [0, dt]

In [ ]:
# Time-stepping loop
for n in range(2, Nt):
    # Finite difference update (interior points)
    u_next[1:-1] = (2 * u_curr[1:-1] - u_prev[1:-1] + 
                   r**2 * (u_curr[2:] - 2*u_curr[1:-1] + u_curr[:-2]))
    
    # Boundary conditions (fixed ends)
    u_next[0] = 0
    u_next[-1] = 0
    
    # Store every 10th time step for visualization
    if n % 10 == 0:
        u_history.append(u_next.copy())
        time_points.append(n * dt)
    
    # Shift arrays for next iteration
    u_prev = u_curr.copy()
    u_curr = u_next.copy()

print(f"Simulation complete. Stored {len(u_history)} snapshots.")

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Initial condition
ax1 = axes[0, 0]
ax1.plot(x, u_history[0], 'b-', linewidth=2)
ax1.set_xlabel('Position $x$ (m)', fontsize=12)
ax1.set_ylabel('Displacement $u$ (m)', fontsize=12)
ax1.set_title('Initial Displacement ($t = 0$)', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, L)
ax1.set_ylim(-0.15, 0.15)

# Plot 2: Multiple time snapshots
ax2 = axes[0, 1]
n_snapshots = 6
indices = np.linspace(0, len(u_history)-1, n_snapshots, dtype=int)
colors = plt.cm.viridis(np.linspace(0, 1, n_snapshots))

for i, idx in enumerate(indices):
    ax2.plot(x, u_history[idx], color=colors[i], 
             label=f't = {time_points[idx]:.2f} s', linewidth=1.5)

ax2.set_xlabel('Position $x$ (m)', fontsize=12)
ax2.set_ylabel('Displacement $u$ (m)', fontsize=12)
ax2.set_title('Wave Propagation Over Time', fontsize=14)
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, L)
ax2.set_ylim(-0.15, 0.15)

# Plot 3: Space-time diagram (heatmap)
ax3 = axes[1, 0]
u_matrix = np.array(u_history)
t_array = np.array(time_points)

im = ax3.imshow(u_matrix.T, aspect='auto', origin='lower',
                extent=[t_array[0], t_array[-1], 0, L],
                cmap='RdBu', vmin=-0.1, vmax=0.1)
ax3.set_xlabel('Time $t$ (s)', fontsize=12)
ax3.set_ylabel('Position $x$ (m)', fontsize=12)
ax3.set_title('Space-Time Diagram', fontsize=14)
cbar = plt.colorbar(im, ax=ax3)
cbar.set_label('Displacement $u$ (m)', fontsize=10)

# Plot 4: Energy conservation check
ax4 = axes[1, 1]

# Calculate approximate total energy at each stored time
energies = []
for i, u in enumerate(u_history):
    # Kinetic energy: (1/2) * mu * integral(v^2 dx)
    # Potential energy: (1/2) * T * integral((du/dx)^2 dx)
    if i == 0:
        v = initial_velocity(x)
    else:
        v = (u_history[i] - u_history[i-1]) / (time_points[i] - time_points[i-1]) if i > 0 else np.zeros_like(u)
    
    dudx = np.gradient(u, dx)
    KE = 0.5 * np.trapz(v**2, x)
    PE = 0.5 * c**2 * np.trapz(dudx**2, x)
    energies.append(KE + PE)

ax4.plot(time_points, energies, 'g-', linewidth=2)
ax4.set_xlabel('Time $t$ (s)', fontsize=12)
ax4.set_ylabel('Total Energy (arb. units)', fontsize=12)
ax4.set_title('Energy Conservation', fontsize=14)
ax4.grid(True, alpha=0.3)

# Add text about energy variation
energy_variation = (max(energies) - min(energies)) / energies[0] * 100
ax4.text(0.05, 0.95, f'Variation: {energy_variation:.2f}%', 
         transform=ax4.transAxes, fontsize=10,
         verticalalignment='top', 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Analysis and Discussion

### Key Observations

1. **Wave Behavior**: The initial triangular pluck splits into two waves traveling in opposite directions at speed $c$. Upon reaching the fixed boundaries, the waves reflect with a phase inversion.

2. **Standing Waves**: The superposition of incident and reflected waves creates standing wave patterns. The solution can be decomposed into normal modes:
   $$\omega_n = \frac{n\pi c}{L}, \quad n = 1, 2, 3, \ldots$$

3. **Energy Conservation**: The finite difference scheme conserves energy to within numerical precision, validating the stability of our implementation.

4. **CFL Condition**: With $r < 1$, the numerical domain of dependence contains the physical domain of dependence, ensuring stability.

### Physical Applications

- **Musical Instruments**: Guitar strings, piano wires, violin strings
- **Structural Engineering**: Cable vibrations in bridges
- **Seismology**: Wave propagation in fault lines

### Extensions

This basic model can be extended to include:
- Damping: $\frac{\partial^2 u}{\partial t^2} + \gamma \frac{\partial u}{\partial t} = c^2 \frac{\partial^2 u}{\partial x^2}$
- Variable tension or density
- External forcing (driven oscillations)
- Multiple dimensions (membranes, plates)